### 言語モデルのN-gramとは

- 言語モデル  
単語の並びに対して、その発生確率を返すもの。通常はたんｇの出現頻度にもとづいて言語モデルが算出される。
- N-gramモデル  
言語モデルの一種で「単語の発生確率がその前のN-1個の単語にのみ依存する」と仮定したモデル。  
（機械学習等に使うテキストの特徴量のことをN-gramという事もある）

例）  
N=3のときの、3-gramモデルは、単語の出現確率が、その前の2（=3-1）個の単語にのみ依存すると仮定したモデル。  
- 3-gramを求める  

$$
P("[BOS]"　"古く"　"から"　"人"　"が"　"居住"　"する"　"[EOS]") = \\
P("[BOS]","古く") \times P("から" | "[BOS]","古く") \times P("人" | "古く","から") \times P("が" | "から","人") \times P("居住" |  "人","が") \times P("する" | "が","居住") \times P("[EOS]" | "居住","する")
$$

単語「人」が出現する確率に注目すると

$$
P("人" | "古く","から")= \\
\displaystyle \frac{ コーパスの中で単語の「古く」「から」が、この順で現れたあとに「人」が現れる回数 }{ コーパスの中で単語の「古く」「から」が、この順で現れる回数 } 
$$

として計算する事ができます。  
同様に全ての単語に対して3-gramを計算しておくと

$$
P("[BOS]" "古く" "から" "人" "が"　"居住"　"する"　"[EOS]") \\
P("[BOS]" "古く" "が" "人" "から"　"居住"　"する"　"[EOS]")
$$

のどちらが自然な文かを比較できる。


### N-gramを計算してみる

- nltkライブラリをインストールする  
Pythonで自然言語処理を行うために便利な機能を持つライブラリ

In [1]:
!pip3 install nltk

  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl


#### sqlite3接続用、今回はパスを通してスクリプトとして読み込んで見る

In [2]:
# https://www.sejuku.net/blog/66459
# システムに関する処理をまとめたライブラリのsysを読み込む
import sys
# 下記でライブラリを読み込めるパス一覧を表示できる。ここにパスを書き込むと異なる階層からライブラリを読み込む事が可能となる。
print(sys.path)
# sys.path.append("相対パス")でsys.pathに追加、ここではディレクトリまでを指定する
sys.path.append("src")

import sqlitedatastore as datastore

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/vagrant/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/vagrant/.local/lib/python3.6/site-packages/IPython/extensions', '/home/vagrant/.ipython']


In [3]:
# 文内の単語のを取得するための関数を作成
def find_xs_in_y(xs, y):
    return [x for x in xs if y['begin'] <= x['begin'] and x['end'] <= y['end']]
'''
リスト内包表記
for x in xs:
    if y['begin'] <= x['begin'] and x['end'] <= y['end']:
        return x

xs変数に格納されているアノテーションのリストからyアノテーションの内側に存在するものだけを取り出す関数
'''

"\nリスト内包表記\nfor x in xs:\n    if y['begin'] <= x['begin'] and x['end'] <= y['end']:\n        return x\n\nxs変数に格納されているアノテーションのリストからyアノテーションの内側に存在するものだけを取り出す関数\n"

### 言語モデルの作成

In [9]:
from nltk.lm import Vocabulary
from nltk.lm.models import MLE
# MLE(Maximum Likelihood Estimator):言語モデルを作成するライブラリ
from nltk.util import ngrams

#言語モデルの作成
def create_language_model(doc_ids, N=3):
    sents = []
    
    # コーパスとして文ごとに単語の原型のリストをsents変数に格納する
    for doc_id in doc_ids:
        all_tokens = datastore.get_annotation(doc_id, 'token')
        
        for sent in datastore.get_annotation(doc_id, 'sentence'):
            tokens = find_xs_in_y(all_tokens, sent)
            # コーパスとして文ごとに単語の原型リストをsents変数に格納する。文頭と文末も単語として追加する。
            sents.append(['__BOS__'] + [token['lemma'] for token in tokens] + ['__EOS__'])
            
    # 作成したコーパスにおける全単語の一覧をvocabに作成
    vocab = Vocabulary([word for sent in sents for word in sent])
    # ngramsを使用し文ごとに3つの単語の並びを取り出す
    text_ngrams = [ngrams(sent, N) for sent in sents]
    '''
    ngramの第一引数には単語リスト、第2引数には関数で指定したデフォルトの3を入れる
    '''
    lm = MLE(order=N, vocabulary=vocab)
    lm.fit(text_ngrams)
    return lm

### 実行してみる

In [7]:
datastore.connect()

# 言語モデルを実行
lm = create_language_model(datastore.get_all_ids(limit=-1), N=3)

# 「古く」「から」という2単語の並びに続く単語の出現確率を言語モデルから呼び出す。
context = ('古く', 'から')
print(context, '->')

# 出現確率を言語モデルから呼び出す
prob_list = [(word, lm.score(word, context)) for word in lm.context_counts(lm.vocab.lookup(context))]
# 確率の大きい順にソート
prob_list.sort(key=lambda x: x[1], reverse=True)
for word, prob in prob_list:
    print('\t{:s}: {:f}'.format(word, prob))
datastore.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<Vocabulary with cutoff=1 unk_label='<UNK>' and 20775 items>
('古く', 'から') ->
	の: 0.089552
	この: 0.044776
	行う: 0.044776
	盛ん: 0.044776
	航空: 0.029851
	音楽: 0.029851
	居住: 0.029851
	広い: 0.014925
	活発: 0.014925
	関係: 0.014925
	海運: 0.014925
	海上: 0.014925
	整える: 0.014925
	文明: 0.014925
	アフガニスタン: 0.014925
	*: 0.014925
	建設: 0.014925
	フォアグラ: 0.014925
	伝わる: 0.014925
	知る: 0.014925
	多様: 0.014925
	中国: 0.014925
	交流: 0.014925
	造成: 0.014925
	北: 0.014925
	ワイン: 0.014925
	使う: 0.014925
	息衝く: 0.014925
	アイスランド: 0.014925
	定住: 0.014925
	民間: 0.014925
	地中: 0.014925
	言語: 0.014925
	下: 0.014925
	モン: 0.014925
	軍備: 0.014925
	水上: 0.014925
	世界: 0.014925
	紛争: 0.014925
	無数: 0.014925
	農業: 0.014925
	利用: 0.014925
	繁栄: 0.014925
	印欧語: 0.014925
	国: 0.014925
	高い: 0.014925
	人: 0.014925
	バスケットボール: 0.014925
	発達: 0.014925
	住民: 0.014925
	自動車: 0.014925
	中東: 0.014925
	通商: 0.014925
